In [1]:
import pandas as pd

In [2]:
import psycopg2
from psycopg2.extras import execute_values

In [ ]:
sattelites_types = {
    'name': 'string',
    'operator': 'string',
    'country_of_operator': 'string',
    'contractor': 'string',
    'country_of_contractor': 'string',
    'purpose': 'string',
    'users': 'string',
    'date_of_launch': 'string' ,
    'class_of_orbit': 'string',
    'orbit_period': 'float64',
    'launch_mass': 'float64',
    'Date of Launch': 'string',
    'expected_lifetime': 'float64',
    'launch_site': 'string',
    'rocket_name': 'string'
}
sattelites_df = pd.read_excel("../data/standardized_data/sattelites.xlsx",dtype=sattelites_types)

In [ ]:
launches_df= pd.read_excel("../data/standardized_data/launches.xlsx")

In [ ]:
rocket_types = {
    'name': 'string',
    'company': 'string',
    'status': 'string',
    'liftoff_thrust Thrust': 'float64',
    'payload_leo': 'float64',
    'stages': 'object',
    'height': 'float64',
    'price': 'float64',
}
rockets_df= pd.read_excel("../data/extracted_data/rockets.xlsx",dtype=rocket_types)

In [6]:
satellite_counts = sattelites_df.groupby(["date_of_launch", "launch_site","rocket_name"]).size().reset_index(name="satellite_count")

In [7]:
dff = satellite_counts[satellite_counts["satellite_count"]>1].sort_values(by="satellite_count", ascending=False)

In [8]:
# 2. Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="space_dw",
    user="root",
    password="root",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

In [9]:
# 3. Prepare Dimension Tables
launch_site_dim = launches_df[['space_port', 'country']].drop_duplicates()
rocket_dim = rockets_df[['name', 'company', 'status', 'liftoff_thrust', 'payload_leo', 'stages', 'height', 'price']].drop_duplicates()
rocket_dim = rocket_dim.where(pd.notnull(rocket_dim), None)
launch_dim = launches_df[['status', 'provider', 'mission','launch_pad']].drop_duplicates()
satellite_dim = sattelites_df[['name', 'country_of_operator', 'operator', 'users', 'purpose', 'class_of_orbit', 'orbit_period', 'launch_mass', 'expected_lifetime', 'contractor', 'country_of_contractor']].drop_duplicates()
satellite_dim = satellite_dim.where(pd.notnull(satellite_dim), None)

In [10]:
date_dim = pd.DataFrame()
date_dim['full_date'] = pd.to_datetime(launches_df['date']).drop_duplicates()
date_dim['year'] = date_dim['full_date'].dt.year
date_dim['month'] = date_dim['full_date'].dt.month
date_dim['day'] = date_dim['full_date'].dt.day

In [11]:
# 4. Insert Dimensions (Bulk Insert)
def bulk_insert_dim(table, df, returning_key):    
    records = [tuple(x) for x in df.to_numpy()]
    columns = ','.join(df.columns)
    placeholders = ','.join(['%s'] * len(df.columns))
    
    insert_sql = f"INSERT INTO {table} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING RETURNING {returning_key};"
    keys = []
    for row in records:
        cur.execute(insert_sql, row)
        res = cur.fetchone()
        if res:
            keys.append(res[0])
    return keys

In [12]:
satellite_dim['country_of_contractor'] = satellite_dim['country_of_contractor'].apply(
    lambda x: x if isinstance(x, list) else [x]
)
satellite_dim['country_of_operator'] = satellite_dim['country_of_operator'].apply(
    lambda x: x if isinstance(x, list) else [x]
)

In [13]:
# Insert dimensions
bulk_insert_dim('dim_launch_site', launch_site_dim, 'launch_site_key')
bulk_insert_dim('dim_rocket', rocket_dim, 'rocket_key')
bulk_insert_dim('dim_satellite', satellite_dim, 'satellite_key')
bulk_insert_dim('dim_launch', launch_dim, 'launch_key')
bulk_insert_dim('dim_date', date_dim, 'date_key')

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [14]:
conn.commit()

In [15]:
# Merge satellites with launches (using rocket, launch_site, date)
sattelites_df['date_of_launch'] = pd.to_datetime(sattelites_df['date_of_launch'])
merged_df = sattelites_df.merge(
    launches_df,
    left_on=['date_of_launch', 'rocket_name', 'launch_site'],
    right_on=['date', 'rocket', 'space_port'],
    how='inner'
)

In [16]:
merged_df

,name,country_of_operator,operator,users,purpose,class_of_orbit,orbit_period,launch_mass,date_of_launch,expected_lifetime,...,rocket_name,date,status,provider,mission,launch_site_y,launch_pad,space_port,country,rocket
0,AAC AIS-Sat1 (Kelpie 1),['United Kingdom'],AAC Clyde Space,Commercial,Earth Observation,LEO,95.00,4.0,2023-01-03,NaN,...,Falcon 9,2023-01-03,Success,SpaceX,Transporter 6 (Dedicated SSO Rideshare),"Space Launch Complex 40 | Cape Canaveral, FL, USA",Space Launch Complex 40,Cape Canaveral,USA,Falcon 9
1,ABS-2,['Multinational'],Asia Broadcast Satellite Ltd.,Commercial,Communications,GEO,1436.03,6330.0,2014-02-06,15.0,...,Ariane 5 ECA,2014-02-06,Success,Arianespace,ABS-2 & Athena-Fidus,"Ariane Launch Area 3 | Guiana Space Centre, Fr...",Ariane Launch Area 3,Guiana Space Centre,French Guiana,Ariane 5 ECA
2,ABS-2A,['Multinational'],Asia Broadcast Satellite Ltd.,Commercial,Communications,GEO,1436.10,1800.0,2016-06-15,15.0,...,Falcon 9,2016-06-15,Success,SpaceX,Eutelsat 117 West B & ABS-2A,"Space Launch Complex 40 | Cape Canaveral, FL, USA",Space Launch Complex 40,Cape Canaveral,USA,Falcon 9
3,ADLER-2,['Austria'],Austrian Space Forum (OEWF)/Spire,Civil,Earth Science,LEO,95.00,11.5,2023-04-15,NaN,...,Falcon 9,2023-04-15,Success,SpaceX,Transporter 7 (Dedicated SSO Rideshare),"Space Launch Complex 4E | Vandenberg SFB, CA, USA",Space Launch Complex 4E,Vandenberg SFB,USA,Falcon 9
4,USA 223,['USA'],National Reconnaissance Office (NRO),Military,Earth Observation,GEO,1436.00,5000.0,2010-11-21,NaN,...,Delta IV Heavy,2010-11-21,Success,United Launch Alliance,NROL-32 (Mentor) (USA-223),"Space Launch Complex 37B | Cape Canaveral, FL,...",Space Launch Complex 37B,Cape Canaveral,USA,Delta IV Heavy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3338,OHS-6,['China'],Zhuhai Orbita Control Engineering Co. Ltd.,Commercial,Earth Observation,LEO,94.60,55.0,2019-09-19,NaN,...,Long March 11,2019-09-19,Success,China Aerospace Science and Technology Corpora...,Zhuhai-1-03,Launch Area 95A | Jiuquan Satellite Launch Cen...,Launch Area 95A,Jiuquan Satellite Launch Center,China,Long March 11
3339,OHS-7,['China'],Zhuhai Orbita Control Engineering Co. Ltd.,Commercial,Earth Observation,LEO,94.60,55.0,2019-09-19,NaN,...,Long March 11,2019-09-19,Success,China Aerospace Science and Technology Corpora...,Zhuhai-1-03,Launch Area 95A | Jiuquan Satellite Launch Cen...,Launch Area 95A,Jiuquan Satellite Launch Center,China,Long March 11
3340,OHS-8,['China'],Zhuhai Orbita Control Engineering Co. Ltd.,Commercial,Earth Observation,LEO,94.60,55.0,2019-09-19,NaN,...,Long March 11,2019-09-19,Success,China Aerospace Science and Technology Corpora...,Zhuhai-1-03,Launch Area 95A | Jiuquan Satellite Launch Cen...,Launch Area 95A,Jiuquan Satellite Launch Center,China,Long March 11
3341,OVS-3,['China'],Zhuhai Orbita Control Engineering Co. Ltd.,Commercial,Earth Observation,LEO,94.60,55.0,2019-09-19,NaN,...,Long March 11,2019-09-19,Success,China Aerospace Science and Technology Corpora...,Zhuhai-1-03,Launch Area 95A | Jiuquan Satellite Launch Cen...,Launch Area 95A,Jiuquan Satellite Launch Center,China,Long March 11


In [17]:
# 6. Insert into Fact Table
fact_records = []
for _, row in merged_df.iterrows():
    cur.execute("SELECT satellite_key FROM dim_satellite WHERE name = %s", (row['name'],))
    satellite_key = cur.fetchone()[0]
    
    cur.execute("SELECT rocket_key FROM dim_rocket WHERE name = %s", (row['rocket_name'],))
    rocket_key = cur.fetchone()[0]
    
    cur.execute("SELECT launch_site_key FROM dim_launch_site WHERE space_port = %s", (row['space_port'],))
    launch_site_key = cur.fetchone()[0]
    
    cur.execute("SELECT date_key FROM dim_date WHERE full_date = %s", (row['date_of_launch'],))
    date_key = cur.fetchone()[0]

    cur.execute("SELECT launch_key FROM dim_launch WHERE status = %s AND provider = %s AND mission = %s", (row['status'], row['provider'], row['mission'],))
    launch_key = cur.fetchone()[0]
    
    fact_records.append((satellite_key, rocket_key, launch_site_key, date_key,launch_key))

In [19]:
# Bulk insert into fact_satellite_launch
execute_values(cur,"INSERT INTO fact_satellite_launch (satellite_key, rocket_key, launch_site_key, date_key, launch_key)VALUES %s", fact_records)

In [20]:
conn.commit()
cur.close()
conn.close()